In [1]:
import os  # For directory and file operations
import numpy as np  # For numerical operations and handling image arrays
import random  # For generating random values for augmentation
from PIL import Image, ImageEnhance  # For image processing and enhancement
from tensorflow.keras.preprocessing.image import load_img  # For loading images
from tensorflow.keras.models import Sequential  # For building the model
from tensorflow.keras.layers import Input, Flatten, Dropout, Dense  # For model layers
from tensorflow.keras.optimizers import Adam  # For optimizer
from tensorflow.keras.applications import VGG16  # For using VGG16 model
from sklearn.utils import shuffle  # For shuffling the data


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_path = '/content/drive/MyDrive/Brain Tumor/Training'
test_path = '/content/drive/MyDrive/Brain Tumor/Testing'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.15
)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    subset='validation'
)


Found 4857 images belonging to 4 classes.
Found 855 images belonging to 4 classes.


# New section

In [5]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(4, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=x)

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=val_generator.samples // val_generator.batch_size
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 1578s 5s/step - accuracy: 0.6586 - loss: 0.8123 - val_accuracy: 0.7111 - val_loss: 0.7150
Epoch 2/10
  1/303 ━━━━━━━━━━━━━━━━━━━━ 35s 119ms/step - accuracy: 0.8125 - loss: 0.3460

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


303/303 ━━━━━━━━━━━━━━━━━━━━ 15s 49ms/step - accuracy: 0.8125 - loss: 0.3460 - val_accuracy: 0.7241 - val_loss: 0.6724
Epoch 3/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 142s 468ms/step - accuracy: 0.8317 - loss: 0.4392 - val_accuracy: 0.7783 - val_loss: 0.5897
Epoch 4/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 14s 46ms/step - accuracy: 0.8750 - loss: 0.2303 - val_accuracy: 0.7689 - val_loss: 0.6072
Epoch 5/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 92s 305ms/step - accuracy: 0.8665 - loss: 0.3691 - val_accuracy: 0.8054 - val_loss: 0.5079
Epoch 6/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 14s 45ms/step - accuracy: 0.8750 - loss: 0.6580 - val_accuracy: 0.8172 - val_loss: 0.4921
Epoch 7/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 93s 306ms/step - accuracy: 0.8691 - loss: 0.3473 - val_accuracy: 0.8302 - val_loss: 0.4716
Epoch 8/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 14s 46ms/step - accuracy: 0.8125 - loss: 0.2459 - val_accuracy: 0.8243 - val_loss: 0.4454
Epoch 9/10
303/303 ━━━━━━━━━━━━━━━━━━━━ 142s 352ms/step - accuracy: 0.8773 - loss: 0.3187 - val_a

In [7]:
model.save('/content/drive/MyDrive/BrainModel.h5')

In [8]:
from google.colab import files
uploaded = files.upload()

Saving Te-me_0090.jpg to Te-me_0090.jpg


In [9]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Replace with your uploaded filename
uploaded_filename = list(uploaded.keys())[0]

# Load and preprocess the image
img = image.load_img(uploaded_filename, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array = img_array / 255.0

# Predict
prediction = model.predict(img_array)
predicted_class = np.argmax(prediction, axis=1)[0]

# Map class index to label
class_labels = list(train_generator.class_indices.keys())
print("Predicted class:", class_labels[predicted_class])


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted class: meningioma
